In [ ]:
'''01_setup_kafka_on_ubuntu'''

'''--------------------------------------------------------------------------------------'''

'''masuk ke dalam server ubuntu'''
'''jadikan user sebagai super user'''
sudo usermod -aG sudo super

'''--------------------------------------------------------------------------------------'''

'''update ubuntu'''
sudo apt update -y

'''install java'''
sudo apt install openjdk-11-jdk -y

'''cek versi java, jika muncul versi java berarti java sudah terinstall'''
java -version

'''--------------------------------------------------------------------------------------'''

'''selanjutnya install confluent'''
'''buat folder Downloads di dalam folder /home'''
sudo mkdir Downloads

'''masuk ke dalam folder Downloads'''
cd home/Downloads

'''download confluent'''
sudo curl -O https://packages.confluent.io/archive/7.5/confluent-7.5.2.tar.gz

'''--------------------------------------------------------------------------------------'''

'''pindahkan confluent ke dalam folder /opt/confluent'''
cd opt
mkdir confluent
cd confluent
sudo mv /home/Downloads/confluent-7.5.2 .

'''extract confluent, menggunakkan command berikut:'''
sudo tar -xvzf confluent-7.5.2.tar.gz
cd confluent-7.5.2

'''set environment variable CONFLUENT_HOME'''
'''masuk ke dalam folder confluent-7.5.2'''
cd /opt/confluent/confluent-7.5.2
'''lalu cari pathnya dengan command'''
pwd
'''akan muncul pathnya, copy pathnya, untuk diset ke dalam environment variable CONFLUENT_HOME'''
/opt/confluent/confluent-7.5.2

'''--------------------------------------------------------------------------------------'''

'''buka file .bashrc'''
vi ~/.bashrc

'''tambahkan baris berikut di dalam file .bashrc'''
export CONFLUENT_HOME=/opt/confluent/confluent-7.5.2
export PATH=$PATH:/usr/lib/jvm/java-11-openjdk-amd64/bin:/opt/confluent/confluent-7.5.2/bin

'''check environment variable CONFLUENT_HOME'''
echo $PATH
'''maka akan muncul path yang telah diset sebelumnya'''

'''check confluent status'''
confluent local services status
'''maka akan muncul status confluent, seharusnya statusnya masih belum running'''

'''--------------------------------------------------------------------------------------'''

'''buka server.properties yang ada di dalam folder /opt/confluent/confluent-7.5.2/etc/kafka'''
cd /opt/confluent/confluent-7.5.2/etc/kafka
vi server.properties

'''uncomment dan sesuaikan baris berikut yang ada di dalam file server.properties'''

'''sebelumnya:'''
# listeners=PLAINTEXT://:9092
'''menjadi:'''
listeners=PLAINTEXT://0.0.0.0:9092

'''sebelumnya:'''
# advertised.listeners=PLAINTEXT://your.host.name:9092
'''menjadi:'''
advertised.listeners=PLAINTEXT://localhost:9092

'''uncomment baris berikut yang ada di dalam file server.properties'''
listener.security.protocol.map=PLAINTEXT:PLAINTEXT,SSL:SSL,SASL_PLAINTEXT:SASL_PLAINTEXT,SASL_SSL:SASL_SSL

'''--------------------------------------------------------------------------------------'''

'''start confluent'''
confluent local services kafka start

'''check lagi confluent status'''
'''kafka dan zookeeper sudah running'''

'''start schema-registry'''
confluent local services schema-registry start

'''check lagi confluent status'''
confluent local services status
'''schema-registry sudah running'''

'''jalankan service kafka connect , menggunakan command berikut:'''
cd /opt/confluent/confluent-7.5.2/bin/
./connect-distributed ../etc/kafka/connect-distributed.properties

'''--------------------------------------------------------------------------------------'''

'''agar kafka connect berjalan otomatistis ketika server di restart, maka jalankan command berikut:'''
sudo vi /etc/systemd/system/kafka-connect.service
'''lalu copy paste script berikut ke dalam file kafka-connect.service'''
[Unit]
Description=Kafka Connect
After=network.target

[Service]
User=super
ExecStart=/opt/confluent/confluent-7.5.2/bin/connect-distributed /opt/confluent/confluent-7.5.2/etc/kafka/connect-distributed.properties
Restart=on-failure

[Install]
WantedBy=multi-user.target

'''lalu jalankan command berikut:'''
sudo systemctl daemon-reload
sudo systemctl start kafka-connect
sudo systemctl enable kafka-connect
sudo systemctl status kafka-connect

'''--------------------------------------------------------------------------------------'''

'''jadi kesimpulannya yang harus dipastikan jalan adalah:'''
'''kafka, zookeeper, schema-registry, dan kafka-connect'''
'''start kafka services, menggunakan command berikut:'''
confluent local services kafka start
'''maka otomatis zookeeper akan ikut jalan'''

'''start schema-registry'''
confluent local services schema-registry start

'''start kafka connect'''
systemctl start kafka-connect

'''--------------------------------------------------------------------------------------'''

'''check lagi confluent status'''
confluent local services status
'''seharusnya semua confluent services kafka, zookeeper, dan schema-registry sudah running'''
Connect is [DOWN]
Control Center is [DOWN]
Kafka is [UP]
Kafka REST is [DOWN]
ksqlDB Server is [DOWN]
Schema Registry is [UP]
ZooKeeper is [UP]

'''check lagi kafka connect status'''
systemctl status kafka-connect
'''seharusnya kafka connect sudah running'''

'''--------------------------------------------------------------------------------------'''

'''test kafka dengan cara hit API berikut:'''
curl -XGET http://localhost:8081

'''kalau muncul response seperti ini, berarti kafka sudah terinstall dengan baik'''
{}

'''--------------------------------------------------------------------------------------'''

'''install jq pretty print untuk mempermudah membaca response'''
'''pastikan python sudah terinstall'''
sudo apt-get install python3-pip

'''install pretty print'''
pip3 install jq

'''test kafka dengan cara hit API berikut:'''
curl -XGET http://localhost:8083/connector-plugins | jq

'''maka akan muncul response seperti berikut:'''
[
  {
    "class": "org.apache.kafka.connect.mirror.MirrorCheckpointConnector",
    "type": "source",
    "version": "7.5.2-ce"
  },
  {
    "class": "org.apache.kafka.connect.mirror.MirrorHeartbeatConnector",
    "type": "source",
    "version": "7.5.2-ce"
  },
  {
    "class": "org.apache.kafka.connect.mirror.MirrorSourceConnector",
    "type": "source",
    "version": "7.5.2-ce"
  }
]

'''--------------------------------------------------------------------------------------'''

'''jika belum muncul response: io.debezium.connector, maka install connector dengan cara berikut:'''
'''masuk ke dalam folder confluent-7.5.2'''
cd /opt/confluent/confluent-7.5.2
'''buat folder connectors'''
sudo mkdir connectors
cd connectors

'''download connector-connector yang kira-kira dibutuhkan'''
'''download connector mysql'''
wget https://repo1.maven.org/maven2/io/debezium/debezium-connector-mysql/2.4.1.Final/debezium-connector-mysql-2.4.1.Final-plugin.tar.gz
'''download connector postgresql'''
wget https://repo1.maven.org/maven2/io/debezium/debezium-connector-postgres/2.4.1.Final/debezium-connector-postgres-2.4.1.Final-plugin.tar.gz
'''download connector sqlserver'''
wget https://repo1.maven.org/maven2/io/debezium/debezium-connector-sqlserver/2.4.1.Final/debezium-connector-sqlserver-2.4.1.Final-plugin.tar.gz

'''--------------------------------------------------------------------------------------'''

'''extract connector, di dalam folder /opt/confluent/confluent-7.5.2/connectors'''
tar -xvzf debezium-connector-mysql-2.4.1.Final-plugin.tar.gz
tar -xvzf debezium-connector-postgres-2.4.1.Final-plugin.tar.gz
tar -xvzf debezium-connector-sqlserver-2.4.1.Final-plugin.tar.gz

'''edit file connect-distributed.properties yang ada di dalam folder /opt/confluent/confluent-7.5.2/etc/kafka'''
cd /opt/confluent/confluent-7.5.2/etc/kafka
vi connect-distributed.properties

'''tambahkan baris berikut di dalam file connect-distributed.properties'''
plugin.path=/usr/share/java,/opt/confluent/confluent-7.5.2/connectors

'''restart confluent'''
confluent local services restart

'''check confluent status'''
curl -XGET http://localhost:8083/connector-plugins | jq

'''maka akan muncul response seperti berikut ini:'''
[
  {
    "class": "io.debezium.connector.mysql.MySqlConnector",
    "type": "source",
    "version": "2.4.1.Final"
  },
  {
    "class": "io.debezium.connector.postgresql.PostgresConnector",
    "type": "source",
    "version": "2.4.1.Final"
  },
  {
    "class": "io.debezium.connector.sqlserver.SqlServerConnector",
    "type": "source",
    "version": "2.4.1.Final"
  },
  ...
]

'''--------------------------------------------------------------------------------------'''

'''artinya connector sudah terinstall dengan baik'''
'''dengan rangkaian langkah di atas, maka kafka sudah terinstall dengan baik'''
'''dan siap digunakan untuk keperluan streaming data'''

'''--------------------------------------------------------------------------------------'''

'''jangan lupa untuk melakukan setting firewall di dalam server ubuntu'''
'''agar bisa diakses dari luar server ubuntu'''

'''masuk ke dalam server ubuntu'''
'''install terlebih dahulu telnetd'''
sudo apt-get update
sudo apt-get install xinetd telnetd

'''aktifkan telnetd'''
sudo /etc/init.d/xinetd start

'''cek status telnetd'''
sudo /etc/init.d/xinetd status

'''maka akan muncul status telnetd'''
● xinetd.service - LSB: Starts or stops the xinetd daemon.
     Loaded: loaded (/etc/init.d/xinetd; generated)
     Active: active (running) since Sun 2023-11-19 13:16:25 UTC; 28s ago
       Docs: man:systemd-sysv-generator(8)
    Process: 15341 ExecStart=/etc/init.d/xinetd start (code=exited, status=0/SUCCESS)
      Tasks: 1 (limit: 2142)
     Memory: 768.0K
        CPU: 77ms
     CGroup: /system.slice/xinetd.service
             └─15353 /usr/sbin/xinetd -pidfile /run/xinetd.pid -stayalive -inetd_compat -inetd_ipv6

Nov 19 13:16:25 dev xinetd[15353]: Reading included configuration file: /etc/xinetd.d/discard [file=/etc/xinetd.d/discard] [line=14]
Nov 19 13:16:25 dev xinetd[15353]: Reading included configuration file: /etc/xinetd.d/discard-udp [file=/etc/xinetd.d/discard-udp] [line=25]
Nov 19 13:16:25 dev xinetd[15353]: Reading included configuration file: /etc/xinetd.d/echo [file=/etc/xinetd.d/echo] [line=14]
Nov 19 13:16:25 dev xinetd[15353]: Reading included configuration file: /etc/xinetd.d/echo-udp [file=/etc/xinetd.d/echo-udp] [line=26]
Nov 19 13:16:25 dev xinetd[15353]: Reading included configuration file: /etc/xinetd.d/servers [file=/etc/xinetd.d/servers] [line=14]
Nov 19 13:16:25 dev xinetd[15353]: Reading included configuration file: /etc/xinetd.d/services [file=/etc/xinetd.d/services] [line=13]
Nov 19 13:16:25 dev xinetd[15353]: Reading included configuration file: /etc/xinetd.d/time [file=/etc/xinetd.d/time] [line=13]
Nov 19 13:16:25 dev xinetd[15353]: Reading included configuration file: /etc/xinetd.d/time-udp [file=/etc/xinetd.d/time-udp] [line=28]
Nov 19 13:16:25 dev xinetd[15353]: 2.3.15.3 started with libwrap loadavg labeled-networking options compiled in.
Nov 19 13:16:25 dev xinetd[15353]: Started working: 0 available services

'''buat agar inetd berjalan otomatis ketika server di restart'''
sudo systemctl daemon-reload
sudo systemctl start xinetd
sudo /lib/systemd/systemd-sysv-install enable xinetd
sudo systemctl status xinetd

'''maka akan muncul status xinetd, seperti yang sudah dijelaskan di atas'''

'''--------------------------------------------------------------------------------------'''

'''jika sudah, maka kita bisa melakukan setting firewall di dalam server ubuntu'''

'''aktifkan firewall di dalam server ubuntu'''
sudo ufw enable

'''aktifkan port 23 untuk telnet'''
sudo ufw allow 23

'''jangan lupa sekalian aktifkan port 22 untuk ssh'''
sudo ufw allow 22

'''jangan lupa sekalian aktifkan port 3307 untuk mysql'''
sudo ufw allow 3307

'''cek status firewall'''
sudo ufw status

'''maka akan muncul status firewall, seperti berikut ini:'''
Status: active

To                         Action      From
--                         ------      ----
23                         ALLOW       Anywhere                  
22                         ALLOW       Anywhere                  
3307                       ALLOW       Anywhere                  
23 (v6)                    ALLOW       Anywhere (v6)             
22 (v6)                    ALLOW       Anywhere (v6)             
3307 (v6)                  ALLOW       Anywhere (v6)  

'''dengan demikian firewall sudah aktif dan port yang kita butuhkan sudah diizinkan oleh firewall ubuntu'''

'''--------------------------------------------------------------------------------------'''

'''siapkan mysql mysql-client-core-8.0, untuk bisa mengakses mysql-source yang ada di luar server ubuntu'''
sudo apt install mysql-client-core-8.0

'''ini digunakan untuk mengakses mysql-source yang ada di luar server ubuntu atau di luar jaringan lokal'''

'''--------------------------------------------------------------------------------------'''

'''jangan lupa set waktu di dalam server ubuntu, agar sesuai dengan waktu di indonesia'''
sudo timedatectl set-timezone Etc/GMT-7
'''ini penting agar logs yang dihasilkan oleh confluent sesuai dengan waktu di indonesia'''

'''--------------------------------------------------------------------------------------'''
